In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from datetime import date

# Data Prep

In [ ]:
df = pd.read_csv('./Data/world_info.csv')

path = './Data/Countries'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

t = pd.date_range(start='1/1/2020', end='27/3/2020')#date.today().strftime("%d/%m/%Y"))
dt = pd.DataFrame(np.arange(0,len(t), 1), index=t, columns=['Day0'])

countries = []
for c in onlyfiles:
    df = pd.read_csv(path+'/'+c)
    df['Dates'] = df['Dates'].astype(str) + ' 2020'
    df['Dates'] = pd.to_datetime(df['Dates'],format='%b %d %Y')
    df.set_index(df['Dates'],inplace=True)
    df = df.merge(dt,how='right',left_index=True, right_index=True,copy=False)
    df.fillna(0,inplace=True)
    df['Country'] = str.split(c,'.')[0]
    df.reset_index(inplace=True, drop=True)
    df = df[['Day0','Country','Dates','Cases','Deaths','Active Cases','Daily Deaths']]
    df['New Cases Last Week']=df['Cases'].diff(periods=7)
    df['New Deaths Last Week']=df['Deaths'].diff(periods=7)
    df['NewCases'] = df['Cases'].diff()
    df['NewDeaths'] = df['Deaths'].diff()
    df['CFR_Current']=df['NewDeaths'].rolling(window=28).sum()/df['NewCases'].shift(13).rolling(window=28).sum()
    df['CFR_Total']=df['Deaths']/df['Cases']
    df.fillna(0,inplace=True)
    countries.append(df)

df = pd.concat(countries)
df = df.sort_values(by='Day0')
df.reset_index(inplace=True,drop=True)        

# parameters
mdeaths = int(df['Deaths'].max())
mcases = int(df['Cases'].max())
mnewcases = int(df['New Cases Last Week'].max())
mnewdeaths = int(df['New Deaths Last Week'].max())
#df.to_csv('./Data/Test/timeseries.csv')
#df

# Area Chart

In [ ]:
for col in ['Cases', 'Deaths','Active Cases','Daily Deaths','New Cases Last Week','New Deaths Last Week']:
    
    fig = px.area(df, x="Day0", y=col, color="Country",)
    fig.update_layout(title=col+' by Country over Time')
    fig.show()

# Line Chart

In [ ]:
for col in ['Cases', 'Deaths','Active Cases','Daily Deaths','New Cases Last Week','New Deaths Last Week']:
    
    fig = px.line(df, x="Day0", y=col, color="Country",)
    fig.update_layout(title=col+' by Country over Time')
    fig.show()

# Animations

In [ ]:
fig = px.scatter(df, x="Cases", y="Deaths", animation_frame="Day0",
           size="Daily Deaths", color="Country", hover_name="Country",animation_group="Country",text="Country",
                 range_x=[0,mcases], range_y=[0,mdeaths],trendline='ols')
fig.update_layout(title='Evolution of Deaths and Cases by Country over Time')
fig.show()

In [ ]:
fig = px.line(df, x="Cases", y="New Cases Last Week",
              color="Country", hover_name="Country",
                 range_x=[500,mcases], range_y=[400,mnewcases],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Cases Last Week and Cases by Country over Time')
plotly.offline.plot(fig, filename = './Saved graphs/CasesNewCasesLogStatic.html', auto_open=False)
fig.show()

In [ ]:
fig = px.line(df, x="Cases", y="New Deaths Last Week",
              color="Country", hover_name="Country",
                 range_x=[500,mcases], range_y=[1,mnewdeaths],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Deaths Last Week and Cases by Country over Time')
fig.show()

In [ ]:
fig = px.line(df, x="Cases", y="New Cases Last Week",animation_frame='Day0',
              color="Country", hover_name="Country",text="Country",
                 range_x=[500,mcases], range_y=[400,mnewcases],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Cases Last Week and Cases by Country over Time')
plotly.offline.plot(fig, filename = './Saved graphs/CasesNewCasesLogAnimated.html', auto_open=False)

fig.show()


In [ ]:
fig = px.line(df, x="Cases", y="New Deaths Last Week",animation_frame='Day0',
              color="Country", hover_name="Country",text="Country",
                 range_x=[500,mcases], range_y=[1,mnewdeaths],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Deaths Last Week and Cases by Country over Time')
fig.show()



# Case Fatality Analysis

In [ ]:
x=np.arange(10,mcases*10,10000)
y=np.arange(10,mdeaths*10,10000)
lethality = [0.005,0.01,0.02,0.04,0.06,0.1]
greyscale = np.flip(np.cumsum([250/len(lethality) for x in lethality]))
lgs = list(zip(lethality,greyscale))
countries = ['China','USA','Germany','Italy','UK','Turkey','Spain','France']


fig = go.Figure()

for l,gs in lgs:
    fig.add_trace(go.Scatter(x=x, y=x*l, mode='lines',
        name=str(round(l*100,1))+'%',text=str(round(l*100,0))+'%',
        marker_color='rgb({},{},{})'.format(gs,gs,gs)
                            )
                 )


for c in countries:
    fig.add_trace(go.Scatter(x=df[df['Country']==c]['Cases'],
                             y=df[df['Country']==c]['Deaths'],
                             mode='lines+markers',text=c,name=c))

fig.update_layout(title="Deaths over Cases incl. Case Fatality Rate (CFR) Iso Lines",
                  xaxis_title="Cases",
                  yaxis_title="Deaths",
                  xaxis_type="log", yaxis_type="log",)
fig.update_xaxes(range=[1, 6])
fig.update_yaxes(range=[0.4, 4.2])  
plotly.offline.plot(fig, filename = './Saved graphs/CFRLines.html', auto_open=False)


fig.show()

In [ ]:
fig = px.line(df[df['Country'].isin(countries)], x="Day0", y="CFR_Total",
              color="Country", hover_name="Country",range_x=[50,85],range_y=[0,0.12])
fig.update_layout(title='Case Fatality Rate (CFR) by Country over Time')
fig.show()


In [ ]:
fig = px.scatter(df[df['Country'].isin(countries)], x="NewCases", y="NewDeaths",
                 color="Country", hover_name="Country",trendline="ols",
                 #log_x=True,log_y=True,#range_x=[200,20000],range_y=[20,2000]
                )
fig.update_layout(title='Case Fatality Rate (CFR) by Country over Time')
fig.show()